# Rede Neural
Nesta tarefa você deve implementar uma rede neural com duas camadas totalmente conectadas. O treinamento e a validação deverão ser executados no CIFAR-10. 


In [4]:
# Configuração inicial

import numpy as np
import matplotlib.pyplot as plt

from rncvc.classifiers.neural_net import NeuralNet

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Permite a recarga automática de arquivos python importados
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ retorna erro relativo """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

Implemente a rede neural na classe `NeuralNet` no arquivo `rncvc/classifiers/neural_net.py`. Os parâmetros (pesos) da rede são armazenados na variável (dicionário) `self.params`, onde as chaves são os nomes (string) das camadas e os valores são numpy arrays com os pesos. A seguir, inicializamos dados sintéticos e uma rede simplificada para guiar o procedimento de desenvolvimento. 

In [21]:
# Criação de um modelo e um conjunto de dados para alguns testes.
# Definimos um seed para que seja possivel a conferência dos resultados.

input_size = 4
hidden_size = 10
num_classes = 3
num_inputs = 5

def init_toy_model():
    np.random.seed(0)
    return NeuralNet(input_size, hidden_size, num_classes, std=1e-1)

def init_toy_data():
    np.random.seed(1)
    X = 10 * np.random.randn(num_inputs, input_size)
    y = np.array([0, 1, 2, 2, 1])
    return X, y

net = init_toy_model()
X, y = init_toy_data()



# Forward: calcule valores de predição para as classes
A função `NeuralNet.loss` utiliza os dados e os parâmetros para calcular as predições para cada classe. Além disso, são calculados: o valor da função de custo e os gradientes dos parâmetros. 

Implemente a primeira parte, que faz a predição das classes. 

In [22]:
scores = net.loss(X, reg=0.1)
print 'Suas predicoes:'
print scores
print
print 'Predicoes corretas:'
correct_scores = np.asarray([
  [-0.81233741, -1.27654624, -0.70335995],
  [-0.17129677, -1.18803311, -0.47310444],
  [-0.51590475, -1.01354314, -0.8504215 ],
  [-0.15419291, -0.48629638, -0.52901952],
  [-0.00618733, -0.12435261, -0.15226949]])
print correct_scores
print

# A diferenca deve ser pequena. Normalmente < 1e-7
print 'Diferenca entre sua implementacao e os valores corretos:'
print np.sum(np.abs(scores - correct_scores))




Suas predicoes:
[[-0.81233741 -1.27654624 -0.70335995]
 [-0.17129677 -1.18803311 -0.47310444]
 [-0.51590475 -1.01354314 -0.8504215 ]
 [-0.15419291 -0.48629638 -0.52901952]
 [-0.00618733 -0.12435261 -0.15226949]]

Predicoes corretas:
[[-0.81233741 -1.27654624 -0.70335995]
 [-0.17129677 -1.18803311 -0.47310444]
 [-0.51590475 -1.01354314 -0.8504215 ]
 [-0.15419291 -0.48629638 -0.52901952]
 [-0.00618733 -0.12435261 -0.15226949]]

Diferenca entre sua implementacao e os valores corretos:
3.68027204961e-08


# Forward: calcule a função de custo

Na mesma função, implemente a segunda parte, calculando os valores da função de custo.

In [28]:
loss, _ = net.loss(X, y, reg=0.1)
correct_loss = 1.30378789133

# deve ser pequena a diferenca, normalmente < 1e-12
print 'Diferença da sua funcao de custo e do custo correto:'
print np.sum(np.abs(loss - correct_loss))

Diferença da sua funcao de custo e do custo correto:
1.79412040779e-13


# Backward

Implemente o restante da função. Aqui os gradientes com respeito às variáveis `W1`, `b1`, `W2`, e `b2` devem ser calculados. 
Com uma implementação correta de todo o processo de backward você deve ser capaz de conferir utilizando o gradiente numérico.

In [49]:
from rncvc.gradient_check import eval_numerical_gradient

# Use o gradiente numérico para verificar sua implementação da etapa backward.
# Se sua implementação estiver correta, a diferença entre os gradientes será 
# inferior a 1e-8 para cada um das camadas de pesos: W1, W2, b1, b2.

loss, grads = net.loss(X, y, reg=0.1)

# as diferenças devem ser pequenas (<1e-8)
for param_name in grads:
    f = lambda W: net.loss(X, y, reg=0.1)[0]
    param_grad_num = eval_numerical_gradient(f, net.params[param_name], verbose=False)
    print '%s erro relativo max: %e' % (param_name, rel_error(param_grad_num, grads[param_name]))

W1 erro relativo max: 3.669858e-09
W2 erro relativo max: 3.440708e-09
b2 erro relativo max: 3.865028e-11
b1 erro relativo max: 2.738422e-09


# Treinando a rede (modelo toy)

Para treinar, utilize o SGD. Preencha os espaços em branco da função NeuralNet.train. Ainda, você precisa implementar a função NeuralNet.predict utilizada para verificar acurácia durante o treinamento. 

Com a implementação completa, executando o código abaixo, você deve ser capaz de obter o valor da função de custo próximo a 0.2.


In [ ]:
net = init_toy_model()
stats = net.train(X, y, X, y,
            learning_rate=1e-1, reg=0.1,
            num_iters=100, verbose=False)

print 'Loss de treinamento: ', stats['loss_history'][-1]

# plotagem dos valores de custo durante o treinamento
plt.plot(stats['loss_history'])
plt.xlabel('iteration')
plt.ylabel('training loss')
plt.title('Training Loss history')
plt.show()

# Carregando o CIFAR-10
Com sua implementação completa (e correta), podemos carregar os dados do CIFAR-10 e usá-los para treinar um modelo de rede neuralem dados reais.  


In [ ]:
from rncvc.data_utils import load_CIFAR10, save_model, load_model

def get_CIFAR10_data():
    """
    Carregando o CIFAR-10 e efetuando pré-processamento para preparar os dados
    para entrada na Rede Neural.     
    """
    # Carrega o CIFAR-10
    cifar10_dir = 'rncvc/datasets/cifar-10-batches-py'    
    X_train, y_train, X_valid, y_valid = load_CIFAR10(cifar10_dir)   

    # Normalizacao dos dados: subtracao da imagem media
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_valid -= mean_image
    
    print X_train.shape
    print X_valid.shape
    
    # Imagens para linhas 
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_valid = X_valid.reshape(X_valid.shape[0], -1)    

    return X_train, y_train, X_valid, y_valid


# Utiliza a funcao acima pra carregar os dados.
X_train, y_train, X_valid, y_valid = get_CIFAR10_data()
print 'Shape dados treinamento: ', X_train.shape
print 'Shape das classes (treinamento): ', y_train.shape
print 'Shape dados validacao: ', X_valid.shape
print 'Shape das classes (validacao): ', y_valid.shape

# Treinando a Rede no CIFAR-10
Para treinar a rede use SGD com momentum. Salve o melhor modelo com a função data_utils.save_model().

In [ ]:
input_size = 32 * 32 * 3
hidden_size = 144
num_classes = 10
net = NeuralNet(input_size, hidden_size, num_classes)


# Treina a rede
stats = net.train(X_train, y_train, X_valid, y_valid,
            num_iters=40000, batch_size=200,
            learning_rate=1e-4, learning_rate_decay=0.95,
            reg=0.5, verbose=True)

# Efetua predicao no conjunto de validacao
val_acc = (net.predict(X_valid) == y_valid).mean()
print 'Acuracia de validacao: ', val_acc

# Salva o modelo da rede treinada
model_path = 'model.pickle'
save_model(model_path, net.params)



# Acompanhando o treinamento

Com os hiperparâmetros definidos anteriormente você provavelmente não obterá bons resultados: a acurácia não deve passar de 30%. 

Uma estratégia para entender o que não está bom durante o treinamento é plotar os valores da função de custo e acurácia durante o treinamento. 

Outra estratégia é visualizar os pesos aprendidos na primeira camada durante o processo de otimização. Normalmente, redes treinadas em dados visuais apresentam padrões estruturais visíveis na primeira camada.


In [ ]:
# Plota a função de custo e acurácia
plt.subplot(2, 1, 1)
plt.plot(stats['loss_history'])
plt.title('Loss history')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.subplot(2, 1, 2)
plt.plot(stats['train_acc_history'], label='train')
plt.plot(stats['val_acc_history'], label='val')
plt.title('Classification accuracy history')
plt.xlabel('Epoch')
plt.ylabel('Clasification accuracy')
plt.show()

In [ ]:
from rncvc.vis_utils import visualize_grid

# Visualiza os pesos da rede

def show_net_weights(net):
    W1 = net.params['W1']
    W1 = W1.reshape(32, 32, 3, -1).transpose(3, 0, 1, 2)
    plt.imshow(visualize_grid(W1, padding=3).astype('uint8'))
    plt.gca().axis('off')
    plt.show()

show_net_weights(net)

# Desafio RNCVC 2017

**Dicas**
* Tune os hiperparâmetros (taxa de aprendizado, número de neurônios, etc). 
* Regularize o modelo: teste o uso de Regularização L2 e Dropout. 
* Use validação cruzada para achar os melhores hiperparâmetros.
* Utilize um esquema mais sofisticado de atualização dos pesos (ex: SGD + Momentum, Nesterov, Adadelta, Adam, RMSProp).
* Teste funções de ativação (ex: ELU, Leaky ReLU) e Batch Normalization. 
* Salve o melhor modelo obtido usando data_utils.save_model() e data_utils.load_model() para abrir.
* O campeão do ano passado conseguiu 62% de acurácia! 


Para o desafio: é <b>fundamental</b> que o trecho abaixo seja executado sem problemas. 


In [ ]:
# Assuma que o modelo são os pesos serializados em disco usando data_utils.save_model()
input_size = 32 * 32 * 3
hidden_size = 500
num_classes = 10
net = NeuralNet(input_size, hidden_size, num_classes)

# Treina a rede
stats = net.train(X_train, y_train, X_valid, y_valid,
            num_iters=1000, batch_size=200,
            learning_rate=1e-4, learning_rate_decay=0.95,
            reg=0.5, verbose=True)

best_model = 'path/to/best/model.pickle'
model = load_model(best_model)
net.params = model

# Retorna um vetor de predição (N x 1), onde N é o número de instâncias
# As classes retornadas aqui devem ser inteiros: [0, 1, 2, ..., C]
# Assuma que X_teste e uma matriz de instancias (N_test, D) 
# Voce nao tera o X_teste oficial, use outro conjunto para validar.
predicted_classes = net.predict(X_train)

Reporte abaixo os valores de acurácia no treino e validação do modelo enviado para o desafio.

In [ ]:
# Treino: 00.00%
# Validacao: 00.00%

In [ ]:
scores1 = [[-2.85, 0.86, 0.28]]

N1 = 1
Y = [1,0]
reg = 0.5

exp_scores2 = np.exp(scores1)
norm =(exp_scores2 / np.sum(exp_scores2, axis=1, keepdims=True)) # [N x K]

# compute softmax loss
loss = -np.sum(np.log(norm[np.arange(N1), Y]))
loss /= N
#loss += 0.5 * reg * (np.sum(W1**2) + np.sum(W2**2))

print norm, loss